<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey%20Model%20Mark%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [35]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [36]:
#Import Even-Strength On-Ice Rates Data
skater_EV_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Rates.csv')
#skater_EV_rates_raw.head()

In [37]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_rates = skater_EV_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF/60','GA/60','xGF/60','xGA/60']]
#skater_EV_rates

In [38]:
skater_EV_rates['EV_TOI'] = skater_EV_rates['TOI']*3600
#skater_EV_rates

In [39]:
#Convert rate/60 to rates/s
skater_EV_rates['GF_EV'] = skater_EV_rates['GF/60']/3600
skater_EV_rates['GA_EV'] = skater_EV_rates['GA/60']/3600
skater_EV_rates['xGF_EV'] = skater_EV_rates['xGF/60']/3600
skater_EV_rates['xGA_EV'] = skater_EV_rates['xGA/60']/3600
#skater_EV_rates

In [40]:
#Drop TOI, GF/60, GA/60, xGF/60 and xGA/60 columns
skater_EV_rates = skater_EV_rates.drop(columns = ['TOI','GF/60','GA/60','xGF/60','xGA/60'])
#skater_EV_rates

In [41]:
#Rename columns
skater_EV_rates = skater_EV_rates.rename(columns = {'Position':'Position_EV','GP':'GP_EV'})
#skater_EV_rates

In [42]:
#Import PP On-Ice Rates Data
skater_PP_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/PP%20On-Ice%20Rates.csv')
#skater_PP_rates_raw.head()

In [43]:
#Reduce PP On-Ice Totals Data
skater_PP_rates = skater_PP_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
#skater_PP_rates

In [44]:
#Convert G±/60 to G/s and TOI to PP_TOI(s)
skater_PP_rates['PP'] = skater_PP_rates['G±/60']/3600
skater_PP_rates['PP_TOI'] = skater_PP_rates['TOI']*3600
#skater_PP_rates

In [45]:
#Drop columns
skater_PP_rates = skater_PP_rates.drop(columns = ['TOI','G±/60'])
#skater_PP_rates

In [46]:
#Rename columns
skater_PP_rates = skater_PP_rates.rename(columns = {'Position':'Position_PP','GP':'GP_PP'})
#skater_PP_rates

In [47]:
#Import SH On-Ice Totals Data
skater_SH_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/SH%20On-Ice%20Rates.csv')
#skater_SH_rates_raw.head()

In [48]:
#Reduce SH On-Ice Totals Data
skater_SH_rates = skater_SH_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
#skater_SH_rates

In [49]:
#Convert TOI to TOI(s) and G±/60 to G± (s)
skater_SH_rates['SH'] = skater_SH_rates['G±/60']/3600
skater_SH_rates['SH_TOI'] = skater_SH_rates['TOI']*3600
#skater_SH_rates

In [50]:
#Drop columns
skater_SH_rates = skater_SH_rates.drop(columns = ['G±/60','TOI'])
#skater_SH_rates

In [51]:
#Rename columns
skater_SH_rates = skater_SH_rates.rename(columns = {'Position':'Position_SH','GP':'GP_SH'})
#skater_SH_rates

In [52]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
#skater_box_totals_raw.head()

In [53]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2']]
#skater_box_totals.head()

In [54]:
skater_box_totals['TOI_tot'] = skater_box_totals['TOI']*3600
#skater_box_totals

In [55]:
skater_box_totals = skater_box_totals.drop(columns = {'TOI'})
#skater_box_totals

In [56]:
#Look at positions to get correct F/D average points
#forwards_table = skater_box_totals.loc[skater_box_totals.Position!= "D"]
#defense_table = skater_box_totals.loc[skater_box_totals.Position== "D"]

In [57]:
#Calculate Mean Points of forwards and defensemen
#Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)
#Defense_mean = defense_table['Points'].mean()
#print(Defensemen_mean)

In [58]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine
#pd.options.mode.chained_assignment = None
#forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
#forwards_table.head(10)

#defense_table['PAA'] = defense_table['Points'] - Defense_mean
#defensemen_table.head(10)

#Add dataframes back together and sort again
#concat_frames = [forwards_table, defense_table]
#skater_box_totals = pd.concat(concat_frames)
#skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

In [59]:
#Combine skater EV and PP stats
skater_EV_PP = pd.merge(skater_EV_rates, skater_PP_rates, how="outer", on=['Player','Season'])
#skater_EV_PP

In [60]:
#Drop duplicate columns
skater_EV_PP = skater_EV_PP.drop(columns = ['Team_y','Position_PP','GP_PP'])
#skater_EV_PP

In [61]:
#Merge EV,PP,SH tables together
skater_EV_PP_SH = pd.merge(skater_EV_PP, skater_SH_rates, how="left", on=["Player", "Season"])
#skater_EV_PP_SH

In [62]:
#Clean up data
skater_EV_PP_SH = skater_EV_PP_SH.drop(columns = ['Team','Position_SH','GP_SH'])
#skater_EV_PP_SH

In [63]:
#Merge EV,PP,SH and Points tables together
full_skater_totals = pd.merge(skater_EV_PP_SH, skater_box_totals, how="left", on=["Player", "Season"])
#full_skater_totals.head()

In [64]:
#Replace NaN with 0
full_skater_totals = full_skater_totals.replace(np.NaN, 0)
#full_skater_totals

In [65]:
#Drop Repeated Columns
full_skater_totals = full_skater_totals.drop(columns = ['Team','Position','GP'])
#full_skater_totals.head(10)


In [66]:
full_skater_totals = full_skater_totals.rename(columns = {'Team_x':'Team','Position_EV':'Position'})
#full_skater_totals

In [67]:
#Look at positions to get correct F/D average points
forwards_table = full_skater_totals.loc[full_skater_totals.Position!= "D"]
defense_table = full_skater_totals.loc[full_skater_totals.Position== "D"]

In [68]:
#Calculate Mean Stats of forwards and defensemen
avg_GF_EV_F = forwards_table['GF_EV'].mean()
avg_GA_EV_F = forwards_table['GA_EV'].mean()
avg_xGF_EV_F = forwards_table['xGF_EV'].mean()
avg_xGA_EV_F = forwards_table['xGA_EV'].mean()
avg_PP_F = forwards_table['PP'].mean()
avg_SH_F = forwards_table['SH'].mean()
avg_G_F = forwards_table['G'].mean()
avg_A1_F = forwards_table['A1'].mean()
avg_A2_F = forwards_table['A2'].mean()
#print(Forwards_mean)
avg_GF_EV_D = defense_table['GF_EV'].mean()
avg_GA_EV_D = defense_table['GA_EV'].mean()
avg_xGF_EV_D = defense_table['xGF_EV'].mean()
avg_xGA_EV_D = defense_table['xGA_EV'].mean()
avg_PP_D = defense_table['PP'].mean()
avg_SH_D = defense_table['SH'].mean()
avg_G_D = defense_table['G'].mean()
avg_A1_D = defense_table['A1'].mean()
avg_A2_D = defense_table['A2'].mean()
#print(Defensemen_mean)

In [70]:
#Add impact for all variables to the tables
pd.options.mode.chained_assignment = None
#forwards
forwards_table['GF_Impact'] = (forwards_table['GF_EV']-avg_GF_EV_F)*forwards_table['EV_TOI']
forwards_table['GA_Impact'] = (forwards_table['GA_EV']-avg_GA_EV_F)*forwards_table['EV_TOI']
forwards_table['xGF_Impact'] = (forwards_table['xGF_EV']-avg_xGF_EV_F)*forwards_table['EV_TOI']
forwards_table['xGA_Impact'] = (forwards_table['xGA_EV']-avg_xGA_EV_F)*forwards_table['EV_TOI']
forwards_table['PP_Impact'] = (forwards_table['PP']-avg_PP_F)*forwards_table['PP_TOI']
forwards_table['SH_Impact'] = (forwards_table['SH']-avg_SH_F)*forwards_table['SH_TOI']
forwards_table['G_Impact'] = (forwards_table['G']-avg_G_F)*forwards_table['TOI_tot']
forwards_table['A1_Impact'] = (forwards_table['A1']-avg_A1_F)*forwards_table['TOI_tot']
forwards_table['A2_Impact'] = (forwards_table['A2']-avg_A2_F)*forwards_table['TOI_tot']
#defense
defense_table['GF_Impact'] = (defense_table['GF_EV']-avg_GF_EV_D)*defense_table['EV_TOI']
defense_table['GA_Impact'] = (defense_table['GA_EV']-avg_GA_EV_D)*defense_table['EV_TOI']
defense_table['xGF_Impact'] = (defense_table['xGF_EV']-avg_xGF_EV_D)*defense_table['EV_TOI']
defense_table['xGA_Impact'] = (defense_table['xGA_EV']-avg_xGA_EV_D)*defense_table['EV_TOI']
defense_table['PP_Impact'] = (defense_table['PP']-avg_PP_D)*defense_table['PP_TOI']
defense_table['SH_Impact'] = (defense_table['SH']-avg_SH_D)*defense_table['SH_TOI']
defense_table['G_Impact'] = (defense_table['G']-avg_G_D)*defense_table['TOI_tot']
defense_table['A1_Impact'] = (defense_table['A1']-avg_A1_D)*defense_table['TOI_tot']
defense_table['A2_Impact'] = (defense_table['A2']-avg_A2_D)*defense_table['TOI_tot']

In [71]:
#Add dataframes back together and sort again
#concat_frames = [forwards_table, defense_table]
#skater_stats = pd.concat(concat_frames)
#skater_stats = skater_stats.sort_index()
#skater_stats

In [72]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Goalie%20Stats.csv')
#goalie_stats_raw.head()

In [73]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','GSAx']]
#goalie_stats.head()

In [74]:
#Add Active / Inactive Column to both data tables
forwards_table['A/I'] = 'Active'
defense_table['A/I'] = 'Active'
goalie_stats['A/I'] = 'Active'

In [75]:
#Put Players on current teams

#NHL API info
#https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
#https://github.com/dword4/nhlapi
#https://gitlab.com/dword4/nhlapi

#API JSON project example
#https://github.com/danmartin25/World_Weather_Analysis/blob/main/WeatherPy.ipynb

#Test all
#data = requests.get('https://statsapi.web.nhl.com/api/v1/configurations').json()
#data

#Different options
#data = requests.get('https://statsapi.web.nhl.com/api/v1/expands').json()
#data

#Empty Dictionary to Hold Rosters
#active_rosters = []

### NOTE: ONLY RUN REQUEST ONCE, THEN COMMENT OUT. INFO WILL STAY IN VARIABLE
#json_api_data = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster').json()

#Variable where data is stored
#json_api_data

#Devils Roster (Index 0, change indext to 1, 2, etc for other teams rosters)
#json_api_data["teams"][0]

#player_name = json_api_data["teams"][0]
#player_team = 
#player_position = #code


#active_rosters.append{{"Player": player_name,
#                       "Team": player_team,
#                       "Postion": player_position}}




In [76]:
#Create Roster for each team
Avalanche_forwards = forwards_table.loc[(forwards_table.Player=="Andrew Cogliano")|(forwards_table.Player=="J.T. Compher")|(forwards_table.Player=="Darren Helm")|(forwards_table.Player=="Nazem Kadri")
|(forwards_table.Player=="Gabriel Landeskog")|(forwards_table.Player=="Artturi Lehkonen")|(forwards_table.Player=="Nathan MacKinnon")|(forwards_table.Player=="Alex Newhook")|(forwards_table.Player=="Valeri Nichushkin")
|(forwards_table.Player=="Logan O'Connor")|(forwards_table.Player=="Mikko Rantanen")|(forwards_table.Player=="Nico Sturm")]
Avalanche_defense = defense_table.loc[(defense_table.Player=="Bowen Byram")|(defense_table.Player=="Jack Johnson")
|(defense_table.Player=="Erik Johnson")|(defense_table.Player=="Cale Makar")|(defense_table.Player=="Josh Manson")|(defense_table.Player=="Devon Toews")]
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Player=="Darcy Kuemper")]
Avalanche_forwards

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,PP_Impact,SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I
84,Alex Newhook,20-21,COL,C,6,268920.0,0.001397,0.000211,0.000978,0.000467,...,201.103670,-128.361052,89.033179,-53.733703,-0.000000,0.000000,-3.080327e+06,-2.167815e+06,-1.106287e+06,Active
85,Alex Newhook,21-22,COL,C,71,3072672.0,0.000669,0.000425,0.000631,0.000631,...,61.582213,-809.439204,-49.609520,-110.382911,176.340056,0.000000,8.924849e+06,9.231782e+06,1.128737e+07,Active
144,Andrew Cogliano,19-20,DAL,C,68,2618892.0,0.000439,0.000436,0.000581,0.000556,...,-551.312505,-660.800368,-173.227660,-290.498182,-0.000000,-438.492822,-2.267219e+07,2.023385e+06,-1.143222e+07,Active
145,Andrew Cogliano,20-21,DAL,C,54,2371248.0,0.000381,0.000619,0.000569,0.000494,...,-637.502863,-163.585888,-183.194348,-407.938100,-0.000000,-206.554364,-1.468914e+07,-1.174220e+07,-1.013708e+07,Active
146,Andrew Cogliano,21-22,COL/S.J,C,74,2835396.0,0.000464,0.000953,0.000517,0.000675,...,-526.004651,749.525816,-368.698704,24.158611,-0.000000,-259.170809,-2.168873e+07,-4.513663e+06,-2.512501e+06,Active
205,Artturi Lehkonen,19-20,MTL,L,70,3181428.0,0.000686,0.000614,0.000719,0.000594,...,116.785691,-237.152582,231.428171,-229.174770,-0.000000,-928.535924,1.000799e+07,1.035217e+07,-1.066969e+07,Active
206,Artturi Lehkonen,20-21,MTL,L,47,1883052.0,0.000411,0.000542,0.000550,0.000531,...,-448.715143,-276.366187,-182.092928,-255.952108,-0.000000,-411.389436,-7.781084e+06,-1.439394e+07,-3.962052e+06,Active
207,Artturi Lehkonen,21-22,COL/MTL,L,74,3179988.0,0.000697,0.000708,0.000789,0.000664,...,152.066030,63.286960,452.155921,-8.238539,198.239217,-658.188973,3.443100e+07,1.470822e+07,5.042102e+06,Active
594,Darren Helm,19-20,DET,C,68,2897460.0,0.000367,0.000522,0.000544,0.000653,...,-819.215870,-481.585309,-296.284152,-39.700581,-0.000000,-598.482223,-4.793478e+06,-1.794358e+07,-5.864517e+06,Active
595,Darren Helm,20-21,DET,C,47,2109780.0,0.000397,0.000522,0.000528,0.000636,...,-532.044981,-350.665429,-250.901743,-64.070903,-0.000000,-444.042286,-1.844170e+07,-1.573839e+07,-6.815580e+06,Active


In [77]:
Avalanche_forwards_2122 = Avalanche_forwards.loc[Avalanche_forwards.Season=="21-22"]
Avalanche_defense_2122 = Avalanche_defense.loc[Avalanche_defense.Season=="21-22"]
Avalanche_goalies_2122 = Avalanche_goalie.loc[Avalanche_goalie.Season=="21-22"]
Avalanche_forwards_2122

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,PP_Impact,SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I
85,Alex Newhook,21-22,COL,C,71,3072672.0,0.000669,0.000425,0.000631,0.000631,...,61.582213,-809.439204,-49.609520,-110.382911,176.340056,0.000000,8.924849e+06,9.231782e+06,1.128737e+07,Active
146,Andrew Cogliano,21-22,COL/S.J,C,74,2835396.0,0.000464,0.000953,0.000517,0.000675,...,-526.004651,749.525816,-368.698704,24.158611,-0.000000,-259.170809,-2.168873e+07,-4.513663e+06,-2.512501e+06,Active
207,Artturi Lehkonen,21-22,COL/MTL,L,74,3179988.0,0.000697,0.000708,0.000789,0.000664,...,152.066030,63.286960,452.155921,-8.238539,198.239217,-658.188973,3.443100e+07,1.470822e+07,5.042102e+06,Active
596,Darren Helm,21-22,COL,C,68,2083320.0,0.000464,0.000739,0.000578,0.000611,...,-386.484290,105.118474,-143.589036,-115.350352,-0.000000,-164.676502,-8.848622e+06,-8.619959e+06,-7.088559e+06,Active
816,Gabriel Landeskog,21-22,COL,L,51,2847852.0,0.001056,0.000581,0.000753,0.000597,...,1156.663691,-307.215290,302.091089,-197.234859,1328.305519,0.000000,7.301799e+07,2.858439e+07,2.706575e+07,Active
885,J.T. Compher,21-22,COL,L,70,3087432.0,0.000700,0.000564,0.000647,0.000642,...,156.216232,-384.517456,1.609374,-76.608350,53.370790,-500.535437,3.115210e+07,2.725712e+06,1.051310e+06,Active
1391,Logan O'Connor,21-22,COL,R,81,3379572.0,0.000608,0.000544,0.000603,0.000594,...,-138.796325,-486.615292,-148.441543,-243.448110,-0.000000,-614.094202,-9.812132e+06,1.481528e+07,-7.055858e+06,Active
1658,Mikko Rantanen,21-22,COL,R,75,4369320.0,0.001214,0.000689,0.000844,0.000642,...,2466.421514,1.997611,864.004579,-108.415796,2086.895457,0.000000,1.448059e+08,1.622937e+08,7.505576e+07,Active
1700,Nathan MacKinnon,21-22,COL,C,65,3878568.0,0.001114,0.000717,0.000833,0.000706,...,1801.541907,109.511244,723.866367,151.558618,1261.930877,0.000000,1.063239e+08,1.461865e+08,6.039923e+07,Active
1707,Nazem Kadri,21-22,COL,C,71,3622932.0,0.001028,0.000733,0.000797,0.000653,...,1370.827774,162.675570,545.328312,-49.640894,1851.350113,-37.530409,8.644181e+07,1.311454e+08,8.979351e+07,Active


In [81]:
Avalanche_forwards_2122['GF_Impact'].sum()

7178.039891271822

In [82]:
Avs_Strength_2122 = 0.5 + (Avalanche_forwards_2122['GF_Impact'].sum()*0.00294) + (Avalanche_forwards_2122['GA_Impact'].sum()*-0.00248) + (Avalanche_forwards_2122['xGF_Impact'].sum()*0.00485) + (Avalanche_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Avalanche_forwards_2122['xGF_Impact'].sum()*0.00485) + (Avalanche_forwards_2122['PP_Impact'].sum()*0.00361) + (Avalanche_forwards_2122['SH_Impact'].sum()*0.00498) + (Avalanche_forwards_2122['G_Impact'].sum()*0.00749)
+ (Avalanche_forwards_2122['A1_Impact'].sum()*0.00771) + (Avalanche_forwards_2122['A2_Impact'].sum()*0.0102) + (Avalanche_defense_2122['GF_Impact'].sum()*0.00444) + (Avalanche_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Avalanche_defense_2122['xGF_Impact']*0.00736) + (Avalanche_defense_2122['xGA_Impact']*-0.00654) + (Avalanche_defense_2122['PP_Impact']*0.00833) + (Avalanche_defense_2122['SH_Impact']*0.00498)
+ (Avalanche_defense_2122['G_Impact'].sum()*0.0132) + (Avalanche_defense_2122['A1_Impact'].sum()*0.00963) + (Avalanche_defense_2122['A2_Impact'].sum()*0.00918) + (Avalanche_goalies_2122['GSAx']*0.00622)
Avs_Strength_2122

41.364297984805496

In [41]:
#Make a Data Table for skaters on each team
Avalanche_skaters = skater_stats.loc[(skater_stats.Team == 'COL')]
Blackhawks_skaters = skater_stats.loc[(skater_stats.Team == 'CHI')]
Blue_Jackets_skaters = skater_stats.loc[(skater_stats.Team == 'CBJ')]
Blues_skaters = skater_stats.loc[(skater_stats.Team == 'STL')]
Bruins_skaters = skater_stats.loc[(skater_stats.Team == 'BOS')]
Canadiens_skaters = skater_stats.loc[(skater_stats.Team == 'MTL')]
Canucks_skaters = skater_stats.loc[(skater_stats.Team == 'VAN')]
Capitals_skaters = skater_stats.loc[(skater_stats.Team == 'WSH')]
Coyotes_skaters = skater_stats.loc[(skater_stats.Team == 'ARI')]
Devils_skaters = skater_stats.loc[(skater_stats.Team == 'N.J')]
Ducks_skaters = skater_stats.loc[(skater_stats.Team == 'ANA')]
Flames_skaters = skater_stats.loc[(skater_stats.Team == 'CGY')]
Flyers_skaters = skater_stats.loc[(skater_stats.Team == 'PHI')]
Hurricanes_skaters = skater_stats.loc[(skater_stats.Team == 'CAR')]
Islanders_skaters = skater_stats.loc[(skater_stats.Team == 'NYI')]
Jets_skaters = skater_stats.loc[(skater_stats.Team == 'WPG')]
Kings_skaters = skater_stats.loc[(skater_stats.Team == 'L.A')]
Knights_skaters = skater_stats.loc[(skater_stats.Team == 'VGK')]
Kraken_skaters = skater_stats.loc[(skater_stats.Team == 'SEA')]
Leafs_skaters = skater_stats.loc[(skater_stats.Team == 'TOR')]
Lightning_skaters = skater_stats.loc[(skater_stats.Team == 'T.B')]
Oilers_skaters = skater_stats.loc[(skater_stats.Team == 'EDM')]
Panthers_skaters = skater_stats.loc[(skater_stats.Team == 'FLA')]
Penguins_skaters = skater_stats.loc[(skater_stats.Team == 'PIT')]
Predators_skaters = skater_stats.loc[(skater_stats.Team == 'NSH')]
Rangers_skaters = skater_stats.loc[(skater_stats.Team == 'NYR')]
Red_Wings_skaters = skater_stats.loc[(skater_stats.Team == 'DET')]
Sabres_skaters = skater_stats.loc[(skater_stats.Team == 'BUF')]
Senators_skaters = skater_stats.loc[(skater_stats.Team == 'OTT')]
Sharks_skaters = skater_stats.loc[(skater_stats.Team == 'S.J')]
Stars_skaters = skater_stats.loc[(skater_stats.Team == 'DAL')]
Wild_skaters = skater_stats.loc[(skater_stats.Team == 'MIN')]

In [42]:
#Make a Data for each team of goalies
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Team == 'COL')]
Blackhawks_goalie = goalie_stats.loc[(goalie_stats.Team == 'CHI')]
Blue_Jackets_goalie = goalie_stats.loc[(goalie_stats.Team == 'CBJ')]
Blues_goalie = goalie_stats.loc[(goalie_stats.Team == 'STL')]
Bruins_goalie = goalie_stats.loc[(goalie_stats.Team == 'BOS')]
Canadiens_goalie = goalie_stats.loc[(goalie_stats.Team == 'MTL')]
Canucks_goalie = goalie_stats.loc[(goalie_stats.Team == 'VAN')]
Capitals_goalie = goalie_stats.loc[(goalie_stats.Team == 'WSH')]
Coyotes_goalie = goalie_stats.loc[(goalie_stats.Team == 'ARI')]
Devils_goalie = goalie_stats.loc[(goalie_stats.Team == 'N.J')]
Ducks_goalie = goalie_stats.loc[(goalie_stats.Team == 'ANA')]
Flames_goalie = goalie_stats.loc[(goalie_stats.Team == 'CGY')]
Flyers_goalie = goalie_stats.loc[(goalie_stats.Team == 'PHI')]
Hurricanes_goalie = goalie_stats.loc[(goalie_stats.Team == 'CAR')]
Islanders_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYI')]
Jets_goalie = goalie_stats.loc[(goalie_stats.Team == 'WPG')]
Kings_goalie = goalie_stats.loc[(goalie_stats.Team == 'L.A')]
Knights_goalie = goalie_stats.loc[(goalie_stats.Team == 'VGK')]
Kraken_goalie = goalie_stats.loc[(goalie_stats.Team == 'SEA')]
Leafs_goalie = goalie_stats.loc[(goalie_stats.Team == 'TOR')]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Team == 'T.B')]
Oilers_goalie = goalie_stats.loc[(goalie_stats.Team == 'EDM')]
Panthers_goalie = goalie_stats.loc[(goalie_stats.Team == 'FLA')]
Penguins_goalie = goalie_stats.loc[(goalie_stats.Team == 'PIT')]
Predators_goalie = goalie_stats.loc[(goalie_stats.Team == 'NSH')]
Rangers_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYR')]
Red_Wings_goalie = goalie_stats.loc[(goalie_stats.Team == 'DET')]
Sabres_goalie = goalie_stats.loc[(goalie_stats.Team == 'BUF')]
Senators_goalie = goalie_stats.loc[(goalie_stats.Team == 'OTT')]
Sharks_goalie = goalie_stats.loc[(goalie_stats.Team == 'S.J')]
Stars_goalie = goalie_stats.loc[(goalie_stats.Team == 'DAL')]
Wild_goalie = goalie_stats.loc[(goalie_stats.Team == 'MIN')]

In [43]:
Avalanche_skaters

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,G±_PP_Impact,G±_SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I
84,Alex Newhook,20-21,COL,C,6,268920.0,0.001397,0.000211,0.000978,0.000467,...,201.103670,-128.361052,89.033179,-53.733703,-0.000000,0.000000,-3.080327e+06,-2.167815e+06,-1.106287e+06,Active
85,Alex Newhook,21-22,COL,C,71,3072672.0,0.000669,0.000425,0.000631,0.000631,...,61.582213,-809.439204,-49.609520,-110.382911,176.340056,0.000000,8.924849e+06,9.231782e+06,1.128737e+07,Active
125,Andre Burakovsky,19-20,COL,L,58,2608200.0,0.001000,0.000644,0.000700,0.000711,...,914.428307,-114.727556,139.014567,116.407818,151.791697,0.000000,3.037136e+07,1.796334e+07,1.667191e+07,Active
126,Andre Burakovsky,20-21,COL,L,53,2430792.0,0.000917,0.000756,0.000767,0.000508,...,649.663512,163.164135,291.611690,-384.420763,696.945619,0.000000,2.468996e+07,1.918575e+07,1.225432e+07,Active
127,Andre Burakovsky,21-22,COL,L,80,4044168.0,0.001047,0.000858,0.000786,0.000714,...,1608.849648,687.110755,563.798088,191.730991,536.573488,0.000000,5.421929e+07,7.337200e+07,4.335791e+07,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,Tyson Jost,20-21,COL,C,54,2265480.0,0.000769,0.000447,0.000722,0.000461,...,271.952544,-546.455244,171.091918,-465.258282,-0.000000,-282.856103,-9.398300e+06,-9.252921e+05,-1.027228e+07,Active
2351,Valeri Nichushkin,19-20,COL,R,65,2751012.0,0.000900,0.000344,0.000722,0.000467,...,689.396632,-946.313064,207.759910,-549.687872,-56.783996,50.092244,8.472893e+06,-7.693116e+06,7.424298e+06,Active
2352,Valeri Nichushkin,20-21,COL,R,55,2292912.0,0.000753,0.000564,0.000856,0.000503,...,237.030334,-285.565703,478.885218,-375.353951,131.386113,-24.372080,-1.186294e+06,-1.208353e+07,3.498092e+06,Active
2353,Valeri Nichushkin,21-22,COL,R,62,3202560.0,0.001028,0.000667,0.000878,0.000628,...,1211.769416,-69.703820,740.037387,-123.945017,941.952428,-92.745091,6.188387e+07,2.404463e+07,3.080825e+07,Active


**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [ ]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_skater_totals.columns

Index(['Player', 'Season', 'Team', 'Position', 'GP', 'TOI_EV', 'G_EV', 'xG_EV',
       'TOI_PP', 'GAA_PP', 'TOI_SH', 'GAA_SH', 'TOI_tot', 'Points', 'PAA',
       'A/I'],
      dtype='object')

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_skater_totals dataframe and the goalie_stats dataframe.

In [ ]:
#Import Skater Box Car Stats
skater_box = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
skater_box

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,...,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,...,20,11,1,6,0,0,5,2,1,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,...,34,56,14,10,0,0,-4,0,0,0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,...,24,46,14,8,1,1,-6,0,0,0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,...,62,54,13,9,0,0,-4,0,0,0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,...,17,49,0,5,0,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,1428.48,5,2,7,14,...,78,67,20,10,4,4,-10,0,0,0
2493,Zdeno Chara,20-21,WSH,D,55,1006.27,2,4,4,10,...,88,33,12,4,2,2,-8,0,0,0
2494,Zdeno Chara,21-22,NYI,D,72,1348.45,2,8,4,14,...,125,46,20,6,5,5,-14,0,0,0
2495,Zemgus Girgensons,19-20,BUF,C,69,951.77,12,4,3,19,...,110,71,5,6,0,0,1,30,43,-13


In [ ]:
#Restrict Data Table to Player, Season, Team, G, A1, A2, iSH, iBLK, iPEN±, FO±
skater_box = skater_box.loc[:,['Player','Season','Team','Position','GP','G','A1','A2','iSF','iBLK','iPEN±','FO±']]
skater_box

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0
2493,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0
2494,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0
2495,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13


In [ ]:
#Import EV Skater Data
skater_EV = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Totals.csv')
skater_EV

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,53.16,53.30,53.14,...,1070.09,52.60,52.78,12.38,84.53,114.87,143.58,-0.18,10.03,90.59
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,54.51,53.65,53.97,...,481.39,26.51,21.58,6.70,54.32,59.26,83.07,4.93,7.68,93.24
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,58.90,59.83,59.35,...,857.55,61.71,41.78,29.17,214.91,307.62,394.70,19.93,9.96,91.61
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,51.47,49.69,49.55,...,276.12,10.93,10.53,-0.29,8.70,-2.64,-4.93,0.40,8.39,90.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,62.20,49.10,48.67,48.25,...,1085.01,42.25,41.58,22.43,-21.33,-43.04,-73.23,0.67,9.85,94.23
2476,Zdeno Chara,20-21,WSH,D,55,817.42,51.49,49.37,49.02,50.39,...,706.99,28.61,26.57,2.03,-9.96,-21.06,11.25,2.04,9.04,91.70
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,53.26,44.88,44.56,43.73,...,1246.03,46.96,55.16,5.92,-123.65,-180.44,-277.79,-8.20,8.92,93.63
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,45.67,47.49,49.21,49.34,...,621.61,25.45,25.86,-4.34,-35.51,-15.13,-16.25,-0.41,6.82,92.66


In [ ]:
#Restrict Data to Goal Differential and Corsi Differential
skater_EV = skater_EV.loc[:,['Player', 'Season', 'Position','G±', 'C±']]
skater_EV

,Player,Season,Position,G±,C±
0,A.J. Greer,21-22,L,0.06,17.36
1,Aaron Ekblad,19-20,D,12.38,143.58
2,Aaron Ekblad,20-21,D,6.70,83.07
3,Aaron Ekblad,21-22,D,29.17,394.70
4,Aaron Ness,19-20,D,-0.29,-4.93
...,...,...,...,...,...
2475,Zdeno Chara,19-20,D,22.43,-73.23
2476,Zdeno Chara,20-21,D,2.03,11.25
2477,Zdeno Chara,21-22,D,5.92,-277.79
2478,Zemgus Girgensons,19-20,C,-4.34,-16.25


In [ ]:
#Merge EV and box car stats together
Game_Score = pd.merge(skater_box, skater_EV)
Game_Score

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0,12.38,143.58
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0,6.70,83.07
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0,29.17,394.70
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0,-0.29,-4.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0,22.43,-73.23
2476,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0,2.03,11.25
2477,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0,5.92,-277.79
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25


In [ ]:
#Add column for player's average game score
Game_Score['GS'] = ((Game_Score['G']*0.75) + (Game_Score['A1']*0.7) + (Game_Score['A2']*0.55) + (Game_Score['iSF']*0.075) + (Game_Score['iBLK']*0.05) + (Game_Score['iPEN±']*0.15) + (Game_Score['FO±']*0.01) + (Game_Score['C±']*0.05) + (Game_Score['G±']*0.15))/ Game_Score['GP']
Game_Score

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36,0.459667
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0,12.38,143.58,0.756134
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0,6.70,83.07,0.806671
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0,29.17,394.70,1.258369
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0,-0.29,-4.93,0.124375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0,22.43,-73.23,0.275044
2476,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0,2.03,11.25,0.264400
2477,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0,5.92,-277.79,0.066299
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25,0.280891


In [ ]:
#Separate Data into forwards and defensemen
forwards_GS = Game_Score.loc[Game_Score.Position!="D"]
defense_GS = Game_Score.loc[Game_Score.Position=="D"]

In [ ]:
#Calculate Average Game Score for forwards and defensemen
avg_GS_forwards = forwards_GS['GS'].mean()
avg_GS_defense = defense_GS['GS'].mean()

In [ ]:
#Calculate GSVA for forwards
pd.options.mode.chained_assignment = None
forwards_GS['GSVA'] = forwards_GS['GS']-avg_GS_forwards
forwards_GS

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS,GSVA
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36,0.459667,0.014407
8,Adam Brooks,19-20,TOR,C,7,0,2,1,2,1,1,0,1.95,-2.68,0.351214,-0.094046
9,Adam Brooks,20-21,TOR,C,11,4,0,1,10,2,1,-3,3.13,3.73,0.470545,0.025285
10,Adam Brooks,21-22,MTL/VGK/WPG,C,25,2,1,0,14,6,0,-11,2.24,-29.75,0.091540,-0.353720
11,Adam Erne,19-20,DET,L,56,2,3,0,63,25,-6,7,-21.29,-138.97,-0.024946,-0.470206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,Zack MacEwen,20-21,VAN,C,34,1,0,1,31,9,-2,0,-0.68,-25.54,0.070471,-0.374789
2473,Zack MacEwen,21-22,PHI,C,75,3,3,3,82,32,-9,-3,-13.76,-221.36,-0.010160,-0.455420
2474,Zack Smith,19-20,CHI,C,50,4,5,2,43,18,-7,-14,-3.10,-79.20,0.122200,-0.323060
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25,0.280891,-0.164369


In [ ]:
forwards_GS.loc[forwards_GS.Player=='Auston Matthews']

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS,GSVA
218,Auston Matthews,19-20,TOR,C,70,47,19,14,290,60,10,92,19.58,185.69,1.366307,0.921047
219,Auston Matthews,20-21,TOR,C,52,41,15,10,222,47,9,33,25.06,148.83,1.512125,1.066865
220,Auston Matthews,21-22,TOR,C,73,60,26,20,348,62,4,153,28.36,467.33,1.823979,1.378719


In [ ]:
avg_GS_forwards

0.4452599914524993

Things for next steps
1. Add goalie table.
2. Separate Players to teams.
3. Create matchups
4. Determine Log Loss for Teams
5.